In [1]:
import pandas as pd

data = pd.read_csv('/kaggle/input/6000-indian-food-recipes-dataset/IndianFoodDatasetCSV.csv')

data.head()

ModuleNotFoundError: No module named 'pandas'

<h3 style="color:violet">Data Exploration</h1>

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6871 entries, 0 to 6870
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Srno                    6871 non-null   int64 
 1   RecipeName              6871 non-null   object
 2   TranslatedRecipeName    6871 non-null   object
 3   Ingredients             6865 non-null   object
 4   TranslatedIngredients   6865 non-null   object
 5   PrepTimeInMins          6871 non-null   int64 
 6   CookTimeInMins          6871 non-null   int64 
 7   TotalTimeInMins         6871 non-null   int64 
 8   Servings                6871 non-null   int64 
 9   Cuisine                 6871 non-null   object
 10  Course                  6871 non-null   object
 11  Diet                    6871 non-null   object
 12  Instructions            6871 non-null   object
 13  TranslatedInstructions  6871 non-null   object
 14  URL                     6871 non-null   object
dtypes: i

In [3]:
data.isnull().sum()

Srno                      0
RecipeName                0
TranslatedRecipeName      0
Ingredients               6
TranslatedIngredients     6
PrepTimeInMins            0
CookTimeInMins            0
TotalTimeInMins           0
Servings                  0
Cuisine                   0
Course                    0
Diet                      0
Instructions              0
TranslatedInstructions    0
URL                       0
dtype: int64

In [4]:
data[data.duplicated(['TranslatedInstructions'], keep=False)]

,Srno,RecipeName,TranslatedRecipeName,Ingredients,TranslatedIngredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Course,Diet,Instructions,TranslatedInstructions,URL
309,328,Four Cheese and Spinach Stuffed Mushrooms Recipe,Four Cheese and Spinach Stuffed Mushrooms Recipe,12 Button mushrooms - (button or crimini varie...,12 Button mushrooms - (button or crimini varie...,10,50,60,4,Continental,Appetizer,Vegetarian,To prepare Four Cheese and Spinach Stuffed Mus...,To prepare Four Cheese and Spinach Stuffed Mus...,https://www.archanaskitchen.com/four-cheese-an...
310,329,Roasted Vegetable Pasta Primavera Recipe,Roasted Vegetable Pasta Primavera Recipe,12 Button mushrooms - (button or crimini varie...,12 Button mushrooms - (button or crimini varie...,10,50,60,4,Continental,Appetizer,Vegetarian,To prepare Four Cheese and Spinach Stuffed Mus...,To prepare Four Cheese and Spinach Stuffed Mus...,https://www.archanaskitchen.com/weekly-lunch-b...


As we can see <b>Translatedinstructions</b> has same recipe for 2 datapoints. By manually looking into the recipe and the <b>RecipeName</b> the datapoint with <b>Srno 329</b> is the repeated and should be dropped.

In [5]:
data.drop_duplicates(subset=['TranslatedInstructions'], inplace=True)

In [7]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenization: Converting words into integers
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['TranslatedInstructions'])

total_words = len(tokenizer.word_index) + 1

# Creating input sequences
input_sequences = []
for line in data['TranslatedInstructions']:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Padding sequences and creating predictors and label
max_sequence_length = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

# Defining a generator function for batch processing
def sequence_generator(input_sequences, labels, batch_size):
    num_batches = len(input_sequences) // batch_size
    while True:
        for batch in range(num_batches):
            start = batch * batch_size
            end = (batch + 1) * batch_size
            yield X[start:end, :], y[start:end, :]

# Model definition
model = Sequential()
model.add(Embedding(total_words, 100))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2, activation='relu'))
model.add(Dense(total_words, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model using fit_generator
batch_size = 32
num_epochs = 50  
steps_per_epoch = len(input_sequences) // (batch_size)

model.fit_generator(
    sequence_generator(X, y, batch_size),
    epochs=num_epochs,
    steps_per_epoch=steps_per_epoch,
    verbose=1
)

# Generating a recipe
def generate_recipe(seed_text, next_words=10):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Example usage
generated_recipe = generate_recipe("Blend onions")
print(generated_recipe)


Epoch 1/50


/tmp/ipykernel_26/1078034460.py:63: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


6330/6330 [==============================] - 527s 82ms/step - loss: 5.6468 - accuracy: 0.1266
Epoch 2/50
6330/6330 [==============================] - 482s 76ms/step - loss: 4.6133 - accuracy: 0.2128
Epoch 3/50
6330/6330 [==============================] - 482s 76ms/step - loss: 4.0547 - accuracy: 0.2608
Epoch 4/50
6330/6330 [==============================] - 479s 76ms/step - loss: 3.6871 - accuracy: 0.2899
Epoch 5/50
6330/6330 [==============================] - 479s 76ms/step - loss: 3.3987 - accuracy: 0.3109
Epoch 6/50
6330/6330 [==============================] - 482s 76ms/step - loss: 3.1655 - accuracy: 0.3291
Epoch 7/50
6330/6330 [==============================] - 482s 76ms/step - loss: 2.9675 - accuracy: 0.3473
Epoch 8/50
1070/6330 [====>.........................] - ETA: 6:39 - loss: 2.8180 - accuracy: 0.3640

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6330/6330 [==============================] - 481s 76ms/step - loss: 2.8070 - accuracy: 0.3638
Epoch 9/50
6330/6330 [==============================] - 482s 76ms/step - loss: 2.6606 - accuracy: 0.3820
Epoch 10/50
6330/6330 [==============================] - 482s 76ms/step - loss: 2.5392 - accuracy: 0.3981
Epoch 11/50
6330/6330 [==============================] - 482s 76ms/step - loss: 2.4317 - accuracy: 0.4127
Epoch 12/50
6330/6330 [==============================] - 479s 76ms/step - loss: 2.3374 - accuracy: 0.4275
Epoch 13/50
6330/6330 [==============================] - 481s 76ms/step - loss: 2.2553 - accuracy: 0.4404
Epoch 14/50
6330/6330 [==============================] - 482s 76ms/step - loss: 2.1739 - accuracy: 0.4547
Epoch 15/50
6330/6330 [==============================] - 482s 76ms/step - loss: 2.0989 - accuracy: 0.4672
Epoch 16/50
6330/6330 [==============================] - 481s 76ms/step - loss: 2.0303 - accuracy: 0.4779
Epoch 17/50
6330/6330 [==============================] - 48

In [8]:
generated_recipe = generate_recipe("Blend tomatoes")
print(generated_recipe)

Blend tomatoes ginger garlic toor dal coriander paste and dry roast it


In [9]:
model.save('/kaggle/working/recipe_generation_model.h5')

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
